# Chat and RAG

In [4]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

In [3]:
caminhos = [
    "arquivos/Explorando o Universo das IAs com Hugging Face.pdf",
    "arquivos/Explorando a API da OpenAI.pdf"
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [ ]:
diretorio = '/tmp/chat_retrieval'

embeddings_model = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

# Create structure chat

In [7]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [10]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr')
)

In [11]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
chat_chain.invoke({'query': pergunta})

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma plataforma que oferece modelos de inteligência artificial, datasets e outros recursos relacionados a IA. Para acessá-lo, você pode criar uma conta no site do Hugging Face e explorar os modelos, datasets e comunidade disponíveis na plataforma. Além disso, para acessar modelos restritos, você pode solicitar acesso seguindo as instruções específicas de cada modelo.'}

# Modify prompt chain

In [13]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(''' 
    Utilize o contexto fornecido para responder a pergunta ao final.
    Se você não sabe a resposta, apenas diga que não sabe e não tente inventar a resposta.
    Utilize três frase no máquimo, mantenha a resposta concisa.
                                            
    Contexto: {context}
                                            
    Pergunta: {question}
                                            
    Resposta:
''')

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt': chain_prompt},
    return_source_documents=True
)

In [16]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
chat_chain.invoke({'query': pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma plataforma que oferece modelos de IA e datasets. \nPara acessá-lo, é necessário criar uma conta e solicitar acesso aos modelos desejados. \nA comunidade do Hugging Face oferece suporte e recursos para facilitar o uso das bibliotecas de IA.',
 'source_documents': [Document(metadata={'doc_id': 1, 'page': 1, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}, page_content='Explorando o Universo das IAs com Hugging Face\nConteúdo\n01. O que é Hugging Face? 5\nAfinal, o que é Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nÉ tudo aberto mesmo? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\nComo usaremos o Hugging Face? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6\n02. A plataforma Hugging Face 7\nModelos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\nPágina

### Stuff

In [17]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece modelos de inteligência artificial, datasets e espaços para colaboração e comunidade. Para acessá-lo, você pode criar uma conta no site do Hugging Face e explorar os recursos disponíveis, como modelos pré-treinados, datasets e interação com a comunidade através de opções como artigos de blog, cursos, servidor de Discord, Forum e GitHub oficial. Além disso, para acessar modelos restritos, você pode solicitar acesso seguindo as instruções específicas para cada modelo.


### Map Reduce

In [18]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma plataforma de inteligência artificial que oferece uma variedade de recursos, como modelos pré-treinados, datasets e espaços de colaboração para processamento de linguagem natural. Para acessá-lo, você pode visitar o site oficial da Hugging Face em https://huggingface.co/ e criar uma conta na plataforma. Uma vez cadastrado e logado, você terá acesso aos diversos recursos disponíveis, como modelos de IA, ferramentas de processamento de linguagem natural e comunidade de desenvolvedores.


### Refine

In [19]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face is a platform that offers various resources related to artificial intelligence, including pre-trained models, datasets, and collaboration spaces. One of the key features of Hugging Face is its active and engaged community. By accessing the platform's homepage and logging in, you can interact with other community members, follow profiles and organizations, and stay updated on relevant posts.

Additionally, the platform provides a range of community options such as blog articles, course pages, access to the Discord server, forum, and official GitHub. These options allow users to engage with the community, share knowledge, and participate in discussions about artificial intelligence.

The libraries provided by Hugging Face are extremely practical. If you find yourself doing something in a "very manual" way, you can refer to the documentation or community to understand if there is a pre-built function or tool to help you. This can streamline your workflow and make your AI deve